### CONFIGURATION

In [17]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
import copy
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
# Hartwig imports
sys.path.append("/Users/peterpriestley/hmf/repos/scripts/analysisscripts") 
import analyseVCF as aVCF
import venn as vn

In [19]:
#CHROM SLICING
minChromFrac = 0#17.443476
maxChromFrac = 26#17.491846

#minChromFrac = 17.689695
#maxChromFrac = 17.73896

In [32]:
# BED FILE
BED_PATH = "/Users/peterpriestley/hmf/analyses/70-30sample/truthSetNew/"
BED_FILE_NAME = "na12878-na24385-somatic-truth-regions.bed"
#GIAB_ID = "NA12878"
#BED_PATH = "/Users/peterpriestley/hmf/analyses/giabTruthsets/"
#BED_FILE_NAME = "NA12878_GIAB_highconf_IllFB-IllGATKHC-CG-Ion-Solid_ALLCHROM_v3.2.2_highconf.bed"

# TRUTH SET
SAMPLE_NAMES_TRUTH = {'NA12878':'70-30truth'}
VCF_PATH_TRUTH = "/Users/peterpriestley/hmf/analyses/70-30sample/truthSetNew/"
VCF_FILE_NAME_TRUTH = "na12878-na24385-somatic-truth.vcf"
#SAMPLE_NAMES_TRUTH = {'INTEGRATION':'70-30truth'}
#VCF_PATH_TRUTH = BED_PATH
#VCF_FILE_NAME_TRUTH = "NA12878_GIAB_highconf_IllFB-IllGATKHC-CG-Ion-Solid_ALLCHROM_v3.2.2_highconf.vcf"


# COMBINED VCF CONFIG
VCF_SAMPLE = "CPCT11111111"
VCF_PATH = "/Users/peterpriestley/hmf/analyses/70-30sample/161110_preliminary1.10regression/"
VCF_FILE_NAME = VCF_SAMPLE + "R_"+ VCF_SAMPLE + "T_merged_somatics.vcf"
#VCF_FILE_NAME = "fpT4OptimisationFAIL.vcf"
#SAMPLE_NAMES = {'CPCT11111111T':'freebayes'}
SAMPLE_NAMES = {VCF_SAMPLE + 'T.mutect':'mutect', \
                VCF_SAMPLE + 'T.freebayes':'freebayes', \
                'TUMOR.strelka':'strelka', \
                'TUMOR.varscan':'varscan'}

#VCF_SAMPLE = "GIAB12878-neoprep"
#VCF_SAMPLE = "12878T"

#VCF_PATH = "/Users/peterpriestley/hmf/analyses/2016SEP-OCTTestRuns/"
#VCF_FILE_NAME = "160918_GIAB12878-neoprep_HTW5FCCXX_NewFilters.filtered_variants.vcf"
#VCF_FILE_NAME = "GIABanalysis09-prep1A.filtered_variants.vcf"
#SAMPLE_NAMES = {VCF_SAMPLE:VCF_SAMPLE}

### Functions

In [21]:
def filterByChromFrac(df):
    return df[(df.chromFrac > minChromFrac)&(df.chromFrac < maxChromFrac)]

In [22]:
def calculateTruth(df,dfTruth):
    df = pd.merge(df,dfTruth,how='left', left_index=True,right_index=True,suffixes=('', '_Truth'))
    df['hasTP'] = False
    df['hasFP'] = False
    for columnName in list(df):
        if columnName.endswith('allele') and not columnName.startswith('truth'):
            df['hasTP'] = (df['hasTP']) | ((df[columnName[:-6]+'indelDiff'] == df['truth_indelDiff']) \
                    & (~pd.isnull(df['truth_indelDiff']) & (df['variantType'] == 'INDEL'))) |((df[columnName] == df['truth_allele']) \
                    & (df['variantType'] == 'SNP'))
            df['hasFP'] = (df['hasFP']) | ((df[columnName[:-6]+'indelDiff'] != df['truth_indelDiff']) \
                    & (df['variantType'] == 'INDEL') & (df[columnName[:-6]+'indelDiff'] != '')& (~pd.isnull(df['truth_allele']))) |((df[columnName] != df['truth_allele']) \
                    & (df['variantType'] == 'SNP') & (df[columnName] != '')& (~pd.isnull(df['truth_allele'])))
    df['Truth'] = (df['hasTP']) &  (df['hasFP'] == False)
    return df

In [23]:
def calcuatePrecisionSensivityMatrix(df):
    outputdata = []
    for columnName in list(df):
        if columnName.endswith('allele') & ~columnName.endswith('_allele'):
            myCaller = columnName[:-6]
            variantTypes = df[(df[myCaller+'allele'] != '')].variantType.unique()
            for variantType in variantTypes:
                truePositives = len(df[(df[myCaller+'allele'] != '') & (df['Truth'] == True) &(df['variantType'] == variantType)])
                positives = len(df[(df[myCaller+'allele'] != '')&(df['variantType'] == variantType)])
                truthSet = len(dfTruth[dfTruth['variantType'] == variantType]) 
                falseNegatives = truthSet - truePositives
                if positives > 0 and truthSet > 0:
                    outputdata.append([variantType, myCaller, truthSet,truePositives,positives-truePositives, falseNegatives, \
                                   round(truePositives/float(positives),4),round(truePositives/float(truthSet),4)])
    
    outputDF = pd.DataFrame(outputdata)
    outputDF.columns = (['variantType','caller','truthSet','truePositives','falsePositives','falseNegatives','precision','sensitivity'])
    return outputDF.sort_values(['variantType','caller'])

<h3> Load VCFs and Prepare DF

In [24]:
## LOAD BED
bed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)
cum_sum =0
for row in bed:
    cum_sum += -int(row[1]) + int(row[2])
print cum_sum

reading BED file
Bed File Loaded
2333895411


In [26]:
## LOAD TRUTH SET VCF
truthBed = copy.deepcopy(bed)
dfTruth = aVCF.loadVaraintsFromVCF(VCF_PATH_TRUTH,VCF_FILE_NAME_TRUTH, \
                                   SAMPLE_NAMES_TRUTH,"Mix-in Truth Set",True,True,truthBed)
dfTruth = filterByChromFrac(dfTruth)
dfTruth.rename(columns={'70-30truthallele':'truth_allele','70-30truthindelDiff':'truth_indelDiff','70-30truthindelPos':'truth_indelPos'}, inplace=True)
dfTruth = dfTruth[['chrom','pos','variantType','ref','truth_allele','truth_indelDiff','truth_indelPos']]
dfTruth = dfTruth.set_index(['chrom','pos'])

reading vcf file: na12878-na24385-somatic-truth.vcf
reading VCF File line: 100000
reading VCF File line: 200000
reading VCF File line: 300000
reading VCF File line: 400000
reading VCF File line: 500000
reading VCF File line: 600000
reading VCF File line: 700000
reading VCF File line: 800000
reading VCF File line: 900000
reading VCF File line: 1000000
reading VCF File line: 1100000
Number variants loaded: 1101868


In [ ]:
# LOAD SAMPLE VCF + match to truth set
sampleBed = copy.deepcopy(bed)
dfProd = aVCF.loadVaraintsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE,True,True,sampleBed)
dfProd = filterByChromFrac(dfProd)
dfProd = dfProd.set_index(['chrom','pos'])
dfProd = calculateTruth(dfProd,dfTruth)

reading vcf file: CPCT11111111R_CPCT11111111T_merged_somatics.vcf
reading VCF File line: 100000
reading VCF File line: 200000


In [ ]:
dfProd[dfProd.chromFrac<1.2].head(10)

In [30]:
dfProd['dbSNP'] = (dfProd['id'].str.contains('rs'))
dfProd['COSMIC'] = (dfProd['id'].str.contains('cosm'))

<h3> SLICE ANALYSIS

In [ ]:
# SLICE SETTINGS
#SLICE_VCF_PATH = "/Users/peterpriestley/hmf/70-30slice/"
#SLICE_VCF_FILE_NAME = "somatics.merged.vcf"
#SLICE_SAMPLE_NAMES = SAMPLE_NAMES 

In [ ]:
#sliceBed = copy.deepcopy(bed)

In [ ]:
#dfSlice = aVCF.loadVaraintsFromVCF(SLICE_VCF_PATH,SLICE_VCF_FILE_NAME,SLICE_SAMPLE_NAMES,VCF_SAMPLE,True,True,sliceBed)
#dfSlice = filterByChromFrac(dfSlice)
#dfSlice = dfSlice.set_index(['chrom','pos'])
#dfSlice = calculateTruth(dfSlice,dfTruth)

In [ ]:
#dfTruth.head(1)

In [ ]:
#dfTruth.xs('180883292', level=1).head(1)

### PRECISION + SENSITIVITY|

In [31]:
outputDF = calcuatePrecisionSensivityMatrix(dfProd)
outputDF

,variantType,caller,truthSet,truePositives,falsePositives,falseNegatives,precision,sensitivity
6,INDEL,freebayes,96385,38971,534,57414,0.9865,0.4043
1,INDEL,strelka,96385,47973,277,48412,0.9943,0.4977
3,INDEL,varscan,96385,62484,535,33901,0.9915,0.6483
5,SNP,freebayes,1005483,702259,364,303224,0.9995,0.6984
4,SNP,mutect,1005483,953178,3946,52305,0.9959,0.9480
0,SNP,strelka,1005483,939862,1323,65621,0.9986,0.9347
2,SNP,varscan,1005483,897861,839,107622,0.9991,0.8930


In [ ]:
dftemp = dfProd.reset_index()
len(dftemp[(dftemp.numCallers>=0)&(dftemp.variantType=="MIXED")])

In [ ]:
#pd.set_option('display.max_rows', 100)
#dftemp[(dftemp.vennSegment.str.contains('freebayes'))&(dftemp.variantType == "INDEL") &(dftemp.Truth == False)].head(700)

In [ ]:
dfProd[(dfProd.Truth==False)&(dfProd.variantType=='INDEL')&(dfProd.freebayesallele !='')]

In [ ]:
#dfTemp = dfProd[(dfProd['filter']!= "str10")]
#outputDF = calcuatePrecisionSensivityMatrix(dfTemp)
#outputDF

In [ ]:
#dfTemp = dfProd[(dfProd.vennSegment != 'FilteredInAll')&((dfProd['filter']== "NoReadCounts")|(dfProd['filter']== "PASS")|(dfProd['filter']== ".")|(dfProd['filter']== "MMQSdiff")|(dfProd['filter']== "VarMMQS") \
#                |(dfProd['filter']== "RefMMQS")|(dfProd['filter']== "VarMMQS,MMQSdiff")|(dfProd['filter']== "RefMMQS,VarMMQS") \
#               |(dfProd['filter']== "RefAvgRL")|(dfProd['filter']== "RefMMQS,VarMMQS,MMQSdiff"))]
#outputDF = calcuatePrecisionSensivityMatrix(dfTemp)
#outputDF

In [ ]:
dfTemp = dfProd[(dfProd['filter']== "PASS")]
outputDF = calcuatePrecisionSensivityMatrix(dfTemp)
outputDF

In [ ]:
dftemp = dfProd.reset_index()
pd.pivot_table(dftemp, values='pos', index=['vennSegment'], aggfunc='count')

In [16]:
dftemp = dfProd[(dfProd['variantType']!= "MIXED")].reset_index()
pd.pivot_table(dftemp, values='pos', index=['variantType','chrom'], columns=['vennSegment'], aggfunc='count').fillna(0)

vennSegment        Intersection  freebayes  freebayes-mutect  mutect  strelka  strelka-freebayes  strelka-freebayes-mutect  strelka-mutect  strelka-varscan  strelka-varscan-freebayes  strelka-varscan-mutect  varscan  varscan-freebayes  varscan-freebayes-mutect  varscan-mutect
variantType chrom                                                                                                                                                                                                                                                                   
INDEL       1               0.0      201.0               0.0     0.0    193.0               71.0                       0.0             0.0           1126.0                     2533.0                     0.0   1014.0              466.0                       0.0             0.0
            10              0.0      130.0               0.0     0.0    130.0               31.0                       0.0             0.0            680.0                     1486.0                     0.0    630.0              248.0                       0.0             0.0
            11              0.0      109.0               0.0     0.0    144.0               39.0                       0.0             0.0            669.0                     1352.0                     0.0    683.0              221.0                       0.0             0.0
            12              0.0      131.0               0.0     0.0    141.0               49.0                       0.0             0.0            713.0                     1469.0                     0.0    638.0              261.0                       0.0             0.0
            13              0.0      103.0               0.0     0.0     74.0               42.0                       0.0             0.0            536.0                     1255.0                     0.0    517.0              203.0                       0.0             0.0
            14              0.0       98.0               0.0     0.0    102.0               31.0                       0.0             0.0            467.0                      978.0                     0.0    464.0              198.0                       0.0             0.0
            15              0.0       81.0               0.0     0.0     58.0               27.0                       0.0             0.0            397.0                      857.0                     0.0    345.0              124.0                       0.0             0.0
            16              0.0       68.0               0.0     0.0     81.0               34.0                       0.0             0.0            383.0                      690.0                     0.0    333.0              128.0                       0.0             0.0
            17              0.0       67.0               0.0     0.0     80.0               30.0                       0.0             0.0            390.0                      744.0                     0.0    308.0              125.0                       0.0             0.0
            18              0.0       75.0               0.0     0.0     79.0               37.0                       0.0             0.0            456.0                     1036.0                     0.0    427.0              170.0                       0.0             0.0
            19              0.0       57.0               0.0     0.0     59.0               22.0                       0.0             0.0            253.0                      505.0                     0.0    264.0               81.0                       0.0             0.0
            2               0.0      227.0               0.0     0.0    231.0               98.0                       0.0             0.0           1233.0                     2721.0                     0.0   1063.0              463.0                       0.0             0.0
            20              0.0       55.0               0.0     0.0     50.0               16.0                       0.0

In [ ]:
dfProd[(dfProd['variantType']== "INDEL")&(dfProd['Truth']== True)&(dfProd['filter']== "PASS")].head(10)

In [ ]:
#PRECISION + SENSITIVITY by caller - SLICE
outputDF = calcuatePrecisionSensivityMatrix(dfSlice)
outputDF

# False Negative Analysis

In [ ]:
pd.options.display.max_rows = 200
caller = "varscan"
dfProd[(dfProd.Truth == True) &(dfProd[caller+'allele'] != '') & (dfProd.variantType == 'INDEL')][['ref',caller+'indelDiff',caller+'QS',caller+'AF','numCallers']]

In [ ]:
dfSlice[(dfSlice['Truth'] == True) & (dfSlice['freebayesallele'] != '') & (dfSlice.variantType == 'INDEL')]

In [ ]:
unfilteredBed = copy.deepcopy(bed)

In [ ]:
# SETTINGS
myCaller = 'varscan'
UNFILTERED_VCF_PATH = "/Users/peterpriestley/hmf/70-30slice/"
if myCaller == 'varscan':
    UNFILTERED_VCF_FILE_NAME = "varscan.snp.vcf"
    UNFILTERED_SAMPLE_NAMES = {"TUMOR":myCaller}
elif myCaller == 'freebayes':
    UNFILTERED_VCF_FILE_NAME = "freebayes.somatic.vcf"
    UNFILTERED_SAMPLE_NAMES = {'CPCT11111111T':myCaller}  
elif myCaller == 'strelka':
    UNFILTERED_VCF_FILE_NAME = "/strelka/results/all.somatic.snvs.vcf"
    UNFILTERED_SAMPLE_NAMES = {"TUMOR":myCaller}
elif myCaller == 'mutect':
    UNFILTERED_VCF_FILE_NAME = "mutect.vcf"
    UNFILTERED_SAMPLE_NAMES = {'CPCT11111111T':myCaller}


dfUnfiltered = aVCF.loadVaraintsFromVCF(UNFILTERED_VCF_PATH,UNFILTERED_VCF_FILE_NAME,UNFILTERED_SAMPLE_NAMES,VCF_SAMPLE,False,True,unfilteredBed)
dfUnfiltered['Truth']=dfUnfiltered.chromPos.isin(dfTruth['chromPos']) 
dfUnfiltered = dfUnfiltered[(dfUnfiltered.chromFrac > minChromFrac)&(dfUnfiltered.chromFrac < maxChromFrac)]

In [ ]:
dfTruth['inUnfiltered'+myCaller]=dfTruth.chromPos.isin(dfUnfiltered[dfUnfiltered[myCaller+'allele'] != '']['chromPos'])
dfTruth['inSlice']=dfTruth.chromPos.isin(dfSlice['chromPos'])

In [ ]:
#False Positives - pre-somatic
dfTruth[['inSlice','pos','inUnfiltered'+myCaller,'variantType']].groupby(['variantType','inUnfiltered'+myCaller]).agg('count')


In [ ]:
dfUnfiltered['inSlice'+myCaller]=dfUnfiltered.chromPos.isin(dfSlice[dfSlice[myCaller+'allele'] != '']['chromPos'])
dfUnfiltered['inSlice']=dfUnfiltered.chromPos.isin(dfSlice['chromPos'])

In [ ]:
dfUnfiltered[['inSlice'+myCaller,'pos','Truth','variantType','filter']].groupby(['variantType','inSlice'+myCaller,'Truth','filter']).agg('count')


In [ ]:

dfUnfiltered[(dfUnfiltered['Truth'] == True)&
            (dfUnfiltered['inSlice'+myCaller] == True)& 
             (dfUnfiltered.variantType == 'SNP') &
             (dfUnfiltered['filter'] != 'PETE')]['pos']

In [ ]:
#FALSE NEGATIVES IN UNFILTERED
dfUnfiltered[(dfUnfiltered['inSlice'+myCaller] == False)& 
             (dfUnfiltered[myCaller+'somaticGT'] != '2')&
             (dfUnfiltered.variantType == 'SNP') &
             (dfUnfiltered['filter'] != 'str10')].sort_values([myCaller+'QS'],ascending=False)

In [ ]:
dfSlice[dfSlice.pos>'37577400'].head(10)

In [ ]:
#FALSE POSITIVES`
dfUnfiltered[(dfUnfiltered['inSlice'+myCaller] == True) & (dfUnfiltered.variantType == 'SNP') &
             (dfUnfiltered['Truth'] == False)].sort_values([myCaller+'QS'],ascending=False)

<h3> SNP

In [ ]:
snpdf = dfProd[(dfProd.variantType == 'SNP')]

In [ ]:
snpdf[['Truth','pos']].groupby(['Truth']).agg('count')

In [ ]:
pd.pivot_table(snpdf, values='pos', index=['numCallers'], columns=['Truth'], aggfunc='count')

In [ ]:
vn.venn([snpdf[snpdf.mutectallele != '']['chromPos'], \
         snpdf[snpdf.strelkaallele != '']['chromPos'], \
        snpdf[snpdf.freebayesallele != '']['chromPos'], \
        snpdf[snpdf.varscanallele != '']['chromPos'] \
        ],['mutect','strelka','freebayes','varscan'],figsize=(6,6))

### <hd3> Indel Venn

In [ ]:
indeldf= dfProd[(dfProd.variantType == 'INDEL')]

In [ ]:
pd.pivot_table(indeldf, values='pos', index=['numCallers'], columns=['Truth'], aggfunc='count')

In [ ]:
vn.venn([indeldf[indeldf.strelkaallele != '']['chromPos'], \
        indeldf[indeldf.freebayesallele != '']['chromPos'], \
        indeldf[indeldf.varscanallele != '']['chromPos'] \
        ],['strelka','freebayes','varscan'],figsize=(6,6))

### <hd3> Read Depth

In [ ]:
for columnName in list(df):
    if columnName.endswith('allelicFreq'):
        df[df[columnName] != ''].hist(column=columnName,by="Truth",bins=40,figsize=(10,5))
        

In [ ]:
df[df.freebayesreadDepth != ''].hist(column="freebayesallelicFreq",by="Truth",bins=30)

In [ ]:
# INDELS BY CALLER BY TRUTH
caller = 'freebayes'
truths = indeldf.Truth.unique()
for truth in truths:
    ser = indeldf[(indeldf.Truth == truth) &(indeldf[caller+'allele'] != '')][caller + 'QS']
    ser = ser.sort_values()
    cum_dist = np.linspace(0.,1.,len(ser))
    ser_cdf = pd.Series(cum_dist, index=ser,name=caller+" INDEL "+str(truth)+": c="+str(ser.count())+" m="+str(round(ser.median(),2)))
    ser_cdf.plot(drawstyle='steps',legend=True,title=caller+" Qual Score CDF - INDELS",figsize=[15,6], xlim=[0,100],ylim=[0,1])

truths = snpdf.Truth.unique()
for truth in truths:
    ser = snpdf[(snpdf.Truth == truth) &(snpdf[caller+'allele'] != '')][caller + 'QS']
    ser = ser.sort_values()
    cum_dist = np.linspace(0.,1.,len(ser))
    ser_cdf = pd.Series(cum_dist, index=ser,name=caller+" SNP "+str(truth)+": c="+str(ser.count())+" m="+str(round(ser.median(),2)))
    ser_cdf.plot(drawstyle='steps',legend=True,title=caller+"Qual Score CDF",figsize=[15,6],xlim=[0,100],ylim=[0,1])

<h3> Allelic Depth

In [ ]:
#Alllele Freq By Caller
for columnName in list(snpdf):
    if columnName.endswith('AF'):
        ser = snpdf[(snpdf[columnName] != '') &(snpdf[columnName[:-2] + 'AF'] > 0.0) \
                    &(snpdf.Truth == True)][columnName[:-2] + 'AF']
        ser = ser.sort_values()
        #ser[len(ser)] = ser.iloc[-1]
        cum_dist = np.linspace(0.,1.,len(ser))
        ser_cdf = pd.Series(cum_dist, index=ser,name=columnName[:-2]+": c="+str(ser.count())+" m="+str(round(ser.median(),2)))
        ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency by Caller (AllelicFreq > 0.0)",figsize=[15,6],xlim=[0,150])

In [ ]:
snpdf = dfProd[(dfProd.variantType == 'SNP')]
truths = [True,False]
for truth in truths:
    #Alllele Freq By Caller
    for columnName in list(snpdf):
        if columnName.endswith('AF'):
            ser = snpdf[(snpdf[columnName] != '') &(snpdf[columnName[:-2] + 'AF'] > 0.0) \
                    &(snpdf.Truth == truth)][columnName[:-2] + 'AF']
            ser = ser.sort_values()
            cum_dist = np.linspace(0.,1.,len(ser))
            ser_cdf = pd.Series(cum_dist, index=ser,name=columnName[:-2]+": c="+str(ser.count())+ " "+ str(truth) +" m="+str(round(ser.median(),2)))
            ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency by Caller (AllelicFreq > 0.0)",figsize=[15,6])

### Allelic Depth Scatter Plot

In [ ]:
caller1 = 'varscan'
caller2 = 'strelka'
tempdf = df[(df[caller1+'allele']!='')&(df[caller2+'allele']!='')]
tempdf.head()
tempdf.plot.scatter(caller1+'allelicFreq',caller2+'allelicFreq' \
                        ,figsize=[6,6] \
                        ,title="Comparison of "+caller1+" and "+caller2+" Allelic Frequency" \
                        ,xlim=[0,1],ylim=[0,1])

<h3> RAINBOW CHARTS

In [ ]:
## RAINBOW
caller = 'freebayes'
plt.scatter(df[(df[caller+'allele']!='')& (df['Truth'] == True)&(df['chrom'] == '22')]['chromFrac'], \
            df[(df[caller+'allele']!='')& (df['Truth'] == True)&(df['chrom'] == '22')][caller+'allelicFreq'], \
            s=10, c='b', marker="s")

In [ ]:
## RAINBOW
caller = 'freebayes'
plt.scatter(df[(df[caller+'allele']!='')& (df['Truth'] == False)&(df['chrom'] == '22')]['chromFrac'], \
            df[(df[caller+'allele']!='')& (df['Truth'] == False)&(df['chrom'] == '22')][caller+'allelicFreq'], \
            s=10, c='b', marker="s")

<h3> Overlapping Analysis

In [ ]:
#NEEDS REFACTORING TO SUPPORT NEW DF FORMAT
'''
sorteddf = df.sort(['chromFrac'], ascending=1)
sorteddf['nextRef'] = sorteddf['ref'].shift(-2)
sorteddf['nextPos'] = sorteddf['pos'].shift(-2)
sorteddf['nextChrom'] = sorteddf['chrom'].shift(-2)
sorteddf['nextVT'] = sorteddf['variantType'].shift(-2)
sorteddf['nextAlleleTumor1'] = sorteddf['alleleTumor1'].shift(-2)
sorteddf['nextAlleleTumor2'] = sorteddf['alleleTumor2'].shift(-2)
sorteddf['nextVennSegment'] = sorteddf['vennSegment'].shift(-2)
sorteddf['lengthRef'] = sorteddf['ref'].str.len()
sorteddf['Dist2Next'] = -sorteddf['pos'].astype(int).diff(-2)

overlapdf = sorteddf[(sorteddf['lengthRef']+0>sorteddf['Dist2Next']) & (sorteddf['Dist2Next']>0)]
overlapdf.head(20)
'''

In [ ]:
#NEEDS REFACTORING TO SUPPORT NEW DF FORMAT
'''
filtereddf = overlapdf[(~overlapdf.vennSegment.isin(['freebayes','varscan','strelka1'])) \
                       & (overlapdf.vennSegment.str.contains('varscan')) \
                       #& (~overlapdf.nextVennSegment.str.contains('varscan')) \
                       & (overlapdf.variantType == 'INDEL') \
                       #& (overlapdf.variantSubType == 'DELETE')
                       & (overlapdf.variantType == overlapdf.nextVT)]
filtereddf[['nextVennSegment','vennSegment','variantType','nextVT','Dist2Next','lengthRef','ref','alleleTumor2', \
            'nextRef','nextAlleleTumor2','variantSubType','chrom','pos']].count()
'''